## Set-Up

In [2]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine, inspect

In [3]:
# Create an engine for the `wine_db.sqlite` database
engine = create_engine("sqlite:///Resources/wine_data.db")

In [4]:
inspector = inspect(engine)
inspector.get_table_names()

['winequalityRed', 'winequalityWhite']

In [5]:
white_wine = engine.execute('SELECT * FROM winequalityWhite').fetchall()

In [6]:
df = pd.DataFrame(white_wine, columns=["wine_id","fixed acidity","volatile acidity", "citric acid",
                                "residual sugar","chlorides","free sulfur dioxide","total sulfur dioxide",
                                "density","pH","sulphates","alcohol","quality"])

In [7]:
# Separate feature variables and target variable
X = df.drop(['quality','wine_id'], axis = 1)
y = df['quality']

In [8]:
# See proportion of good vs bad wines
df['quality'].value_counts()

1    3258
0    1640
Name: quality, dtype: int64

In [9]:
# Normalize feature variables
from sklearn.preprocessing import StandardScaler
X_features = X
scalar = StandardScaler().fit(X)
X = StandardScaler().fit_transform(X)

In [10]:
# Splitting the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=0)

In [11]:
from sklearn.metrics import classification_report
from sklearn.ensemble import ExtraTreesClassifier
model = ExtraTreesClassifier(random_state=1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.67      0.75       441
           1       0.83      0.93      0.88       784

    accuracy                           0.84      1225
   macro avg       0.84      0.80      0.81      1225
weighted avg       0.84      0.84      0.83      1225



In [12]:
import pickle
pkl_filename = "app/white_model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(model,file)

scaler = StandardScaler().fit(X)
pkl_scaled_filename = 'app/scaler_white.pkl'
with open(pkl_scaled_filename, 'wb') as file:
    pickle.dump(scaler,file)


In [13]:
# Function to Load and run model with input scores

def white_model(user_input_array):
    import pickle
    #Load Model
    with open('app/white_model.pkl', 'rb') as file:
        white_model = pickle.load(file)
    with open('app/scaler_white.pkl', 'rb') as file:
        scaler = pickle.load(file)

    X = user_input_array.reshape(1,-1)
    X_scaled = scaler.transform(X)

    # Calculate Wine quality
    y_pred_white = white_model.predict(X_scaled)

    return y_pred_white

In [14]:
test = X_test[1]
answer = y[1]
print(test, answer)

[ 0.17209696  0.31511748 -0.6131152   0.19887157  1.06325748 -0.42975081
  0.50923618  0.45897907 -0.38590965 -0.26155273 -0.58047134] 1


In [15]:
white_model(test)

array([1])

In [19]:
df.head(30)

,wine_id,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,0,7.0,0.27,0.36,20.70,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,1
1,1,6.3,0.30,0.34,1.60,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,1
2,2,8.1,0.28,0.40,6.90,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,1
3,3,7.2,0.23,0.32,8.50,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,1
4,4,7.2,0.23,0.32,8.50,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,1
5,5,8.1,0.28,0.40,6.90,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,1
6,6,6.2,0.32,0.16,7.00,0.045,30.0,136.0,0.9949,3.18,0.47,9.6,1
7,7,7.0,0.27,0.36,20.70,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,1
8,8,6.3,0.30,0.34,1.60,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,1
9,9,8.1,0.22,0.43,1.50,0.044,28.0,129.0,0.9938,3.22,0.45,11.0,1


In [20]:
df.iloc[9:11,:]

,wine_id,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
9,9,8.1,0.22,0.43,1.50,0.044,28.0,129.0,0.9938,3.22,0.45,11.0,1
10,10,8.1,0.27,0.41,1.45,0.033,11.0,63.0,0.9908,2.99,0.56,12.0,0
